# converting fitchpork to np using wtf

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import json
import os

from tf_to_dict import tf_to_dict

class InversePCA(tf.keras.layers.Layer):
    """
    Inverse PCA layer for tensorflow neural network
    
    Usage:
        - Define dictionary of custom objects containing Inverse PCA
        - Use arguments of PCA mean and components from PCA of output parameters for inverse PCA (found in JSON dict)
        
    Example:

    > f = open("pcann_info.json")
    >
    > data = json.load(f)
    >
    > pca_comps = np.array(data["pca_comps"])
    > pca_mean = np.array(data["pca_mean"])
    > 
    > custom_objects = {"InversePCA": InversePCA(pca_comps, pca_mean)}
    > pcann_model = tf.keras.models.load_model("pcann_name.h5", custom_objects=custom_objects)
    
    """
    
    def __init__(self, pca_comps, pca_mean, **kwargs):
        super(InversePCA, self).__init__()
        self.pca_comps = pca_comps
        self.pca_mean = pca_mean
        
    def call(self, x):
        y = tf.tensordot(x, np.float32(self.pca_comps),1) + np.float32(self.pca_mean)
        return y
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'pca_comps': self.pca_comps,
            'pca_mean': self.pca_mean
        })
        return config

class WMSE(tf.keras.losses.Loss):
    """
    Weighted Mean Squared Error Loss Function for tensorflow neural network
    
    Usage:
        - Define list of weights with len = labels
        - Use weights as arguments - no need to square, this is handled in-function
        - Typical usage - defining target precision on outputs for the network to achieve, weights parameters in loss calculation to force network to focus on parameters with unc >> weight.
    
    """
    
    def __init__(self, weights, name = "WMSE",**kwargs):
        super(WMSE, self).__init__()
        self.weights = np.float32(weights)
        
    def call(self, y_true, y_pred):
        loss = ((y_true - y_pred)/(self.weights))**2
        return tf.math.reduce_mean(loss)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'weights': self.weights
        })
        return config

def WMSE_metric(y_true, y_pred):
    metric = ((y_true - y_pred)/(weights))**2
    return tf.reduce_mean(metric)

with open(f'pitchfork_info.pkl', 'rb') as fp:
    pitchfork_info = pickle.load(fp)

## def data dir
repo_dir = '/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/'

data_dir = repo_dir + 'data/'

## load in PCA variables
pca_mean = np.loadtxt(data_dir+'pca_mean_fp.csv')
pca_components = np.loadtxt(data_dir+'pca_components_fp.csv')

## load in WMSE weights
WMSE_weights = np.loadtxt(data_dir+'WMSE_weights_fp.csv')

## new pitchfork_info file:
fitchpork_info = pitchfork_info.copy()

fitchpork_info['custom_objects']['inverse_pca']['pca_comps'] = pca_components.tolist()
fitchpork_info['custom_objects']['inverse_pca']['pca_mean'] = pca_mean.tolist()

custom_objects = {
    "InversePCA": InversePCA(
        fitchpork_info['custom_objects']['inverse_pca']['pca_comps'],
        fitchpork_info['custom_objects']['inverse_pca']['pca_mean'],
    ),
    "WMSE": WMSE(
        fitchpork_info['custom_objects']['WMSE']['weights'],
    ),
}

2026-01-26 09:45:30.006313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-26 09:45:30.017947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-26 09:45:30.021436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-26 09:45:30.031020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-26 09:45:30.975109: W tensorflow/compiler/tf2

In [3]:
model_dir = repo_dir + '/models'
dict_dir = repo_dir + '/model_dicts'

tf_model = tf.keras.models.load_model(
    f'{model_dir}/fitchpork-duo.h5',
    custom_objects = custom_objects
)

wtf_dict = tf_to_dict(tf_model)

with open(f'{model_dir}/fitchpork-duo.json', 'w') as fp:
    json.dump(wtf_dict, fp)

---
wtf: converting tensorflow model to dict!
---
wtf: finding model info...
	found optimiser: Adam
	found learning rate: 0.0010000000474974513
---
wtf: finding model layers...
	found input_1
	found dense
	found dense_1
	found dense_4
	found dense_5
	found dense_6
	found dense_7
	found dense_8
	found dense_2
	found dense_9
	found dense_3
	found dense_10
	found classical_outs
	custom object or layer detected! skipping - you can deal with this manually :^)
---
wtf: populating dict with outbound_layers:
	input_1-->dense
	dense-->dense_1
	dense_1-->dense_4
	dense_4-->dense_5
	dense_5-->dense_6
	dense_6-->dense_7
	dense_7-->dense_8
	dense_8-->dense_9
	dense_2-->dense_3
	dense_9-->dense_10
	dense_3-->classical_outs
	dense_10 has no outbound layers - output layer?
	classical_outs has no outbound layers - output layer?
---
wtf: adding detected network structure to dict:
	input_1
	  |
	  v
	dense
	  |
	  v
	dense_1
	 /\ branch!
	v  v
	--- branch: 0 ---
	dense_1
	  |
	  v
	dense_2
	  |
	  v
	den